<a href="https://colab.research.google.com/github/thai94/d2l/blob/main/5.deep_learning_computation/5_1_layers_and_blocks_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import torch
from torch import nn
from torch.nn import functional as F

In [68]:
class MLP(nn.Module):

  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.out = nn.Linear(256, 10)

  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

In [69]:
X = torch.rand(2, 20)

In [70]:
X.shape

torch.Size([2, 20])

In [71]:
net = MLP()

In [72]:
y = net(X)

In [73]:
print(y.shape)
y

torch.Size([2, 10])


tensor([[ 0.2561, -0.0616,  0.2754,  0.0986,  0.0298, -0.0977, -0.1810,  0.3225,
         -0.0349,  0.1721],
        [ 0.3375,  0.0198,  0.1613,  0.1531,  0.1566, -0.0940, -0.2711,  0.2084,
          0.0517,  0.0891]], grad_fn=<AddmmBackward0>)

In [74]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self._modules[str(idx)] = module
  
  def forward(self, X):
    for block in self._modules.values():
      X = block(X)
    return X

In [75]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0981, -0.0491,  0.0119,  0.0562, -0.0078, -0.0514, -0.0332,  0.0052,
          0.2526,  0.0752],
        [ 0.0169, -0.1741,  0.0738, -0.0126, -0.0145,  0.0655,  0.1295,  0.0722,
          0.0398,  0.0618]], grad_fn=<AddmmBackward0>)

In [76]:
class FixedHiddenMLP(nn.Module):

  def __init__(self):
    
    super().__init__()
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.linear = nn.Linear(20, 20)

  def forward(self, X):

    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

In [77]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3538, grad_fn=<SumBackward0>)

In [78]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
    self.linear = nn.Linear(32, 16)

  def forward(self, X):
    return self.linear(self.net(X))

In [79]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())

In [80]:
chimera(X)

tensor(0.1569, grad_fn=<SumBackward0>)